# Python notebook for pre-processing apical image stack.
# --- UNDER DEVELOPMENT ---
Assumes folder directory structure:
<pre><code>  IMAGING
    image_stacks
    notebooks
    results
</code></pre>
Execute the code sequentially, one block at a time, using &lt;shift-return&gt;.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from skimage import data, color
from skimage.morphology import binary_erosion, binary_dilation
from skimage.morphology import remove_small_objects

from skimage.transform import hough_circle, hough_circle_peaks, resize
from skimage.feature import canny
from skimage.draw import circle_perimeter
from skimage import filters
from skimage import io
from skimage import exposure

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from ipyfilechooser import FileChooser


In [ ]:
#image_stack = "../image_stacks/Mistgcamp-3_0003.tif"
#image_stack = "../image_stacks/MOVEMENT.tif"
#image_stack = "../image_stacks/Movement2.tif"
#image_stack = "../image_stacks/modestmovement.tif"
image_stack = "../image_stacks/lessmovement3.tif"
image_bits = 10


In [ ]:
# look up table for image plot color
#   e.g. coolwarm, jet, plasma, gray
lut = mpl.cm.gray


In [ ]:
# Load picture
images = io.imread(image_stack)
images = np.float32(images/(2.0**image_bits))
fig=plt.figure(figsize=(10,10))
plt.imshow(images[0], norm=None, cmap=lut.name)

In [ ]:
A0 = np.sum(images[0:50], axis=0)/50.0
for n in range(A0.shape[0] - 1): # moving average over every two lines
  A0[n] = (A0[n] + A0[n+1]) / 2.0
#A0 = exposure.equalize_hist(A0)
A0 = filters.gaussian(A0, sigma=1.0)
#fig=plt.figure(figsize=(10,10))
#plt.imshow(A0, norm=None, cmap=lut.name)

A0uint8 = np.uint8(255.0*A0)
#fig=plt.figure(figsize=(10,10))
#plt.imshow(A0uint8, norm=None, cmap=lut.name)

edges = canny(A0uint8, sigma=2.0, low_threshold=0, high_threshold=0.1)
#edges = binary_dilation(edges)
#edges = binary_dilation(edges)
#edges = binary_erosion(edges)
#edges = remove_small_objects(edges, 5)
#large = remove_small_objects(edges, 100)
#edges = edges != large
#fig=plt.figure(figsize=(10,10))
#plt.imshow(edges, norm=None, cmap=lut.name)

# Detect multiple radii
hough_radii = np.arange(4, 8, 1)
#print(hough_radii)
hough_res = hough_circle(edges, hough_radii)
#print(hough_res.shape)

# Select the most prominent circles
accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii, min_xdistance=10, 
                                           min_ydistance=10, total_num_peaks=300, 
                                           threshold=0.001, normalize=False)
#print(radii)

# Draw them
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10,10))
imageA = color.gray2rgb(A0uint8)
imageB = color.gray2rgb(edges)
for center_y, center_x, radius in zip(cy, cx, radii):
    #circy, circx = circle_perimeter(center_y, center_x, radius, shape=image.shape)
    #imageA[circy, circx] = (255,0,0)
    imageA[center_y, center_x] = (255,0,0)
ax.imshow(imageA, norm=None)
#ax[1].imshow(imageB, norm=None)
plt.show()

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

@interact(
  sr=widgets.IntRangeSlider(description='stack range',min=0, max=300, step=1, value=[0,10]), 
  gs=widgets.FloatSlider(description='GAUSSIAN sigma',min=0.0, max=4.0, step=0.1, value=1.0), 
  cs=widgets.FloatSlider(description='CANNY sigma',min=1.0, max=4.0, step=0.1, value=2.0), 
  ct=widgets.IntRangeSlider(description='threshold',min=0, max=100, step=1, value=[10,22]),
  hr=widgets.IntRangeSlider(description='HOUGH radii',min=3, max=25, step=1, value=[5,8]),
  hd=widgets.IntSlider(description='distance',min=5, max=20, step=1, value=10),
  hp=widgets.IntSlider(description='peaks',min=50, max=500, step=10, value=260),
  ht=widgets.FloatSlider(description='threshold',min=0.0, max=1.0, step=0.01, value=0.1))

def f(sr, gs, cs, ct, hr, hd, hp, ht):
  A0 = 1.8*np.sum(images[sr[0]:sr[1]], axis=0)/float(sr[1]-sr[0])
  for n in range(A0.shape[0] - 1): # moving average over every two lines
    A0[n] = (A0[n] + A0[n+1]) / 2.0
  A = filters.gaussian(A0, sigma=gs)
  Auint8 = np.uint8(255.0*A)
  edges = canny(Auint8, sigma=cs, low_threshold=ct[0], high_threshold=ct[1])
  hough_radii = np.arange(hr[0], hr[1], 1)
  hough_res = hough_circle(edges, hough_radii)
  accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii, min_xdistance=hd, 
                                           min_ydistance=hd, total_num_peaks=hp, 
                                           threshold=ht, normalize=False)
  fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10,10))
  imageA = color.gray2rgb(np.uint8(255.0*A0))
  imageB = color.gray2rgb(edges)
  for center_y, center_x, radius in zip(cy, cx, radii):
    imageA[center_y, center_x] = (255,0,0)
  ax.imshow(imageA, norm=None)
  plt.show()    
  return()


In [51]:

# https://github.com/Borda/BIRL/blob/master/bm_experiments/bm_comp_perform.py

import matplotlib.pyplot as plt
import numpy as np
import time
import os

from skimage import data, io
from skimage.transform import resize, warp, AffineTransform
from skimage.color import rgb2gray
from skimage.measure import ransac
from skimage.util import random_noise
from skimage.restoration import denoise_bilateral, denoise_wavelet
from skimage.feature import ORB, match_descriptors

def register_image_pair(idx, path_img_target, path_img_source, path_out):
    """ register two images together

    :param int idx: empty parameter for using the function in parallel
    :param str path_img_target: path to the target image
    :param str path_img_source: path to the source image
    :param str path_out: path for exporting the output
    :return tuple(str,float):
    """
    start = time.time()
    # load and denoise reference image
    img_target = 10.0*io.imread(path_img_target)[0]
    img_target = denoise_wavelet(img_target, wavelet_levels=7, multichannel=False, rescale_sigma=False)

    # load and denoise moving image
    img_source = 10.0*io.imread(path_img_source)[50]
    img_source = denoise_bilateral(img_source, sigma_color=0.05,
                                   sigma_spatial=2, multichannel=False)

    # detect ORB features on both images
    detector_target = ORB(n_keypoints=150)
    detector_source = ORB(n_keypoints=150)
    detector_target.detect_and_extract(img_target)
    detector_source.detect_and_extract(img_source)
    matches = match_descriptors(detector_target.descriptors,
                                detector_source.descriptors)
    print(matches)
    
    # robustly estimate affine transform model with RANSAC
    model, inliers = ransac((detector_target.keypoints[matches[:, 0]],
                       detector_source.keypoints[matches[:, 1]]),
                      AffineTransform, min_samples=25, max_trials=500,
                      residual_threshold=0.95)  # 0.95
    print(inliers)
    
    # warping source image with estimated transformations
    img_warped = warp(img_target, model.inverse, output_shape=img_target.shape[:2])
    path_img_warped = os.path.join(path_out, "result.tif")
    io.imsave(path_img_warped, img_warped)

    # summarise experiment
    execution_time = time.time() - start
    return path_img_warped, execution_time 



In [52]:
register_image_pair(0, 
                    "../image_stacks/Mistgcamp-3_0003.tif", 
                    "../image_stacks/Mistgcamp-3_0003.tif", 
                    "../image_stacks/")

[[  1   7]
 [  3  80]
 [  4 147]
 [ 11  86]
 [ 19  49]
 [ 21  23]
 [ 23  90]
 [ 24  53]
 [ 25 136]
 [ 28  30]
 [ 31  43]
 [ 36  68]
 [ 37 112]
 [ 41  70]
 [ 46  99]
 [ 47  79]
 [ 48 120]
 [ 50  56]
 [ 52 102]
 [ 61 105]
 [ 69  76]
 [ 72 137]
 [ 73  48]
 [ 79   9]
 [ 82 119]
 [ 83  78]
 [ 88 123]
 [ 91   0]
 [ 92   5]
 [ 95  52]
 [ 96 142]
 [ 97 126]
 [ 98 106]
 [102  91]
 [106  57]
 [107 131]
 [109  13]
 [110  93]
 [111  26]
 [112 115]
 [119  34]
 [120  25]
 [121  59]
 [124  64]
 [131  47]
 [132  95]
 [135  58]
 [140 100]
 [143  89]
 [146  28]
 [147  33]]
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False]


('../image_stacks/result.tif', 2.5475828647613525)

In [ ]:
#The transforms module uses a (x, y) coordinate convention to be consistent
#with most of the warping literature out there.  But the rest of
#scikit-image uses  a (row, column) convention.

#The following code works for me:

from skimage import io, transform
import numpy as np

image = io.imread('image.jpg')
h, w = image.shape[:2]

rng = np.random.RandomState(0)
xs = rng.randint(0, w - 1, 76)
ys = rng.randint(0, h - 1, 76)

src_pts = np.column_stack([xs, ys])
dst_pts = src_pts

tform = transform.PiecewiseAffineTransform()
tform.estimate(src_pts, dst_pts)

out = transform.warp(image, tform)

import matplotlib.pyplot as plt
plt.imshow(out)
plt.show()

